---
###Objetivo:

Uma analise comparativa de pandas X spark (de preferencia no databricks)

###O que fazer:

Pegar 2 datasets um > 50mb e outro menor

Realizar pre-processamento de dados e analise exploratoria dos dados, usando pandas e pyspark

Identificar quando o panda ou pyspark é melhor

###Datasets escolhidos:

https://www.kaggle.com/datasets/swatikhedekar/exploratory-data-analysis-on-netflix-data

Dataset público com informações sobre série e filmes disponiveis na plataforma de streaming Netflix de 2008 a 2021.

Tamanho do dataset = **3.4 mb**


https://www.kaggle.com/datasets/lasaljaywardena/12-million-company-data

Dataset com informações variadas sobre mais de 12 milhões de empresas no mundo.

Tamanho do dataset = **1.63 gb**

---

###pandas
import pandas as pd

df1 = pd.read_csv("/dbfs/FileStore/shared_uploads/briancamargos@gmail.com/netflix_titles_2021.csv")

###spark

df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/briancamargos@gmail.com/netflix_titles_2021.csv")